In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import string
import csv
from nltk.corpus import stopwords 
from nltk.tokenize import  word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from py_stringmatching import  OverlapCoefficient
from py_stringmatching import  Dice
from py_stringmatching import  Jaccard

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kwgi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Αρχικά θα διαβάσουμε τα αρχεία και θα τα μετατρέψουμε σε dataframes ελέγχοντας για null τιμές και τα μεγέθη των dataframes.Η κωδικοποίηση για το διάβασμα όλων των αρχείων θα είναι cp437 γιατί η utf8 μου έβγαζε error.

In [2]:
# load users and print first 5  users
users = pd.read_csv('BX-Users.csv',low_memory=False,sep=';',encoding='cp437')
print(users.isna().any())
print(users.shape)
users.head()

User-ID     False
Location    False
Age          True
dtype: bool
(278858, 3)


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


Παραπάνω δεν μας πειράζει που υπάρχουν null τιμές στην στήλη "Age".
Σε αυτό το σημείο χρειάζεται η υλοποίηση μιας μεθόδου που θα απομακρύνει κυρίως από τον τίτλο του βιβλίου κάποια σύμβολα που μπερδεύουν το pandas στην δημιουργία το dataframe books.Ο συνδυασμός συμβόλων επιλέχθηκαν εμπειρικά για να αφαιρεθούν.

In [3]:
# transforming file BX-Books.csv to the right format for csv and pandas
def transformCSV(inputfile,outputfile):
    with open(inputfile,encoding='cp437') as fp1,open(outputfile,'w',encoding='cp437') as fp2:
        line = fp1.readline()
        while line:
            if '&amp;' in line:
                line= line.replace('&amp;',' ')
            if ';\\' in line: 
                 line= line.replace(';\\',' ')
            if '"; ' in line: 
                 line= line.replace("'; \'",' ')  
            if ' ; '  in line: 
                 line= line.replace(' ; ',' ')  
            if '"; '  in line: 
                 line= line.replace('"; ',' ')  
            fp2.write(line)
            line = fp1.readline()

In [4]:
# transforming file BX-Book-Ratings.csv to the right format for csv and pandas
transformCSV('BX-Books.csv','BX-Books-r.csv')
# load books print first 5 books and print first 5 books
books = pd.read_csv('BX-Books-r.csv',error_bad_lines=False, low_memory=False,sep=';',encoding='cp437')
# get rid off unnecessary columns 
books = books[['ISBN','Book-Title','Book-Author','Year-Of-Publication']]
print(books.shape)
print(books.isna().any())
books.head()

(271379, 4)
ISBN                   False
Book-Title             False
Book-Author             True
Year-Of-Publication    False
dtype: bool


,ISBN,Book-Title,Book-Author,Year-Of-Publication
0,0195153448,Classical Mythology,Mark P. O. Morford,2002
1,0002005018,Clara Callan,Richard Bruce Wright,2001
2,0060973129,Decision in Normandy,Carlo D'Este,1991
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999


Παραπάνω δεν μας πειράζει που υπάρχουν null τιμές στην στήλη "Book-Author".

In [5]:
# load ratings and print first 5 ratings 
ratings = pd.read_csv('BX-Book-Ratings.csv',low_memory=False,sep=';',encoding='cp437')
print(ratings.shape)
print(ratings.isna().any())
ratings.head()

(1149780, 3)
User-ID        False
ISBN           False
Book-Rating    False
dtype: bool


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


Μέσω μελέτης των δεδομένων παρατήρησα πως υπάρχουν κάποια ratings για βιβλία που δεν υπάρχουν για αυτό το λόγο αφαιρώ αυτά τα ratings και ελέγχω αν συμβαίνει το ίδιο και για τους χρήστες.

In [6]:
# prune ratings with books that don't exist 
ratings = ratings.merge(books[['ISBN']], how = 'inner' , on = 'ISBN' )
print('Pruned ratings size: ' , ratings.shape)
# prune ratings with users that don't exist 
ratings = ratings.merge(users[['User-ID']], how = 'inner' , on = 'User-ID' )
print('Pruned ratings size: ' , ratings.shape)

Pruned ratings size:  (1031175, 3)
Pruned ratings size:  (1031175, 3)


Όπως φαίνεται η παραπάνω υπόθεση που κάναμε ισχύει μόνο για τα βιβλία. Στην συνέχεια αφαιρούνται όλα τα βιβλία που έχουν λιγότερες από 10 βαθμολογίες και όλους οι χρήστες που έχουν βαθμολογήσει λιγότερα από 5 βιβλία η διαδικασία επαναλαμβάνεται μέχρι να μην υπάρξει καμία αλλαγή στα σύνολα δεδομένων.

In [7]:
# intial sizes
ratings_size = ratings.shape
books_size = books.shape
users_size = users.shape

while True: 

    # prune all books that have less than 10 ratings  
    ratings_count = pd.DataFrame(ratings.groupby('ISBN')['Book-Rating'].count())
    ratings_count.rename(columns={'Book-Rating':'counts'},inplace = True)
    ratings_count.reset_index(inplace =True)
    ratings_count = ratings_count[ratings_count['counts'] > 10 ]
    # update ratings
    ratings = ratings.merge(ratings_count[['ISBN']], how = 'inner' , on = 'ISBN' )
    
    # prune all users that have less than 5 ratings 
    ratings_count = pd.DataFrame(ratings.groupby('User-ID')['Book-Rating'].count())
    ratings_count.rename(columns={'Book-Rating':'counts'},inplace = True)
    ratings_count.reset_index(inplace =True)
    ratings_count = ratings_count[ratings_count['counts'] > 5 ]
    # update ratings
    ratings = ratings.merge(ratings_count[['User-ID']], how = 'inner' , on = 'User-ID' )

    if ratings_size == ratings.shape and books_size == books.shape and users_size == users.shape: 
        break
    else:
        ratings_size = ratings.shape
        books_size = books.shape
        users_size = users.shape
        
# update dataframes
books = books.merge(ratings[['ISBN']], how = 'inner' , on = 'ISBN' ).drop_duplicates()
users = users.merge(ratings[['User-ID']],how = 'inner' , on = 'User-ID').drop_duplicates()    

# print new sizes of ratings,users and books  
print('ratings: ',ratings.shape)
print('users: ',users.shape)
print('books: ',books.shape)

ratings:  (385632, 3)
users:  (10404, 3)
books:  (12508, 4)


Όπως μπορείτε να δείτε, το σύνολο δεδομένων "ratings" μειώθηκε στις 385632 εγγραφές, το 'users' στις 10404 και το 'books' στις 12508.Παρακάτω υλοποιείτε μια μέθοδος για να παίρνουμε τον τίτλο του βιβλίου δίνοντας το ISBN του.

In [8]:
def getbooknamefromISBN(ISBN):
    return books.set_index('ISBN').loc[ISBN]['Book-Title']

In [9]:
getbooknamefromISBN(books['ISBN'].sample().tolist()[0])

'The Alibi'

Στην συνέχεια δημιουργούμε ένα πίνακα οφέλους απο τα ratings.

In [10]:
# create the matrix of ISBN and corresponding user ratings
user_book_rating = ratings.pivot_table(index='User-ID', columns='ISBN', values='Book-Rating')
user_book_rating.head()

ISBN,0002251760,000649840X,0006547834,0006550576,0006550789,0007106572,0007110928,0007141076,0007154615,000716226X,...,3791513265,379200027X,3803112133,3822860867,8251800811,842046435X,8433925180,8495501198,8817125539,950491036X
User-ID,,,,,,,,,,,,,,,,,,,,,
17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
user_book_rating.shape

(10404, 12508)

Εδώ αρχίζει η προεπεξεργασία των τίτλων για την δημιουργία λέξεων κλειδιών. Με αυτόν τον κώδικα θέτουμε σε μια καινούρια στήλη "KeyWords" στο σύνολο δεδομένων books όλους τους τίτλους των βιβλίων με πεζά γράμματα.

In [12]:
books['KeyWords'] = books['Book-Title'].str.lower()
books['KeyWords'].head(10)

0                                  the kitchen god's wife
24                                          the testament
355                  beloved (plume contemporary fiction)
496     our dumb century: the onion presents 100 years...
511     new vegetarian: bold and beautiful recipes for...
526                                           wild animus
1748                                             airframe
1911                                             timeline
2197                                     prague : a novel
2215                                          lying awake
Name: KeyWords, dtype: object

Παρακάτω αφαιρούμε τα σημεία στίξης

In [13]:
# remove punctuations from Keywords(Title)
def remove_punctuation(text):
    no_punct = "".join([c for c in text if c not in string.punctuation])
    return no_punct

In [14]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : remove_punctuation(x))
books["KeyWords"].head(10)

0                                   the kitchen gods wife
24                                          the testament
355                    beloved plume contemporary fiction
496     our dumb century the onion presents 100 years ...
511     new vegetarian bold and beautiful recipes for ...
526                                           wild animus
1748                                             airframe
1911                                             timeline
2197                                      prague  a novel
2215                                          lying awake
Name: KeyWords, dtype: object

Στην συνέχεια κάνουμε tokenize.

In [15]:
# tokenize Keywords(Title)
books["KeyWords"] = books["KeyWords"].fillna("").map(word_tokenize)
books["KeyWords"].head(10)

0                              [the, kitchen, gods, wife]
24                                       [the, testament]
355               [beloved, plume, contemporary, fiction]
496     [our, dumb, century, the, onion, presents, 100...
511     [new, vegetarian, bold, and, beautiful, recipe...
526                                        [wild, animus]
1748                                           [airframe]
1911                                           [timeline]
2197                                   [prague, a, novel]
2215                                       [lying, awake]
Name: KeyWords, dtype: object

Έπειτα αφαιρούμε τα stopwords στα Αγγλικά.

In [16]:
# remove stopwords from Keywords 
def remove_stopwords(text):
    words = [w for w in text if w not in stopwords.words('english')]
    return words

In [17]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : remove_stopwords(x))
books["KeyWords"].head(10)

0                                   [kitchen, gods, wife]
24                                            [testament]
355               [beloved, plume, contemporary, fiction]
496     [dumb, century, onion, presents, 100, years, h...
511     [new, vegetarian, bold, beautiful, recipes, ev...
526                                        [wild, animus]
1748                                           [airframe]
1911                                           [timeline]
2197                                      [prague, novel]
2215                                       [lying, awake]
Name: KeyWords, dtype: object

Τέλος έχουμε την επιλογή να χρησιμοποιήσουμε είτε lemmatizer είτε stemmer για να κόψουμε τις καταλήξεις των λέξεων. Εγώ χρησιμοποιώ stemmer γιατί πιστεύω είναι ποιο χρήσιμος λόγο του μικρού μεγέθους των τίτλων.

In [18]:
# Instantiate lemmatizer 
lemmatizer = WordNetLemmatizer()
def word_lemmatizer(text):
    lem_text = " ".join([lemmatizer.lemmatize(i) for i in text])
    return lem_text

In [19]:
# Instantiate Stemmer 
stemmer = PorterStemmer()
def word_stemmer(text):
    stem_text = [stemmer.stem(i) for i in text]
    return stem_text

In [20]:
books["KeyWords"] = books["KeyWords"].apply(lambda x : word_stemmer(x))
books["KeyWords"].head(10)

0                                    [kitchen, god, wife]
24                                            [testament]
355                 [belov, plume, contemporari, fiction]
496     [dumb, centuri, onion, present, 100, year, hea...
511     [new, vegetarian, bold, beauti, recip, everi, ...
526                                         [wild, animu]
1748                                            [airfram]
1911                                            [timelin]
2197                                       [pragu, novel]
2215                                          [lie, awak]
Name: KeyWords, dtype: object

Στην συνέχεια υλοποιώ μια μέθοδο για να παίρνω τα 3 βιβλία με την υψηλότερη βαθμολογία για έναν χρήστη καθώς και όλα τα βιβλία που έχει βαθμολογήσει

In [21]:
# find 3 books' ISBN with the highest rating from the user and return them with all the books user has rated
def get3largest_values(user):
    user_books = user_book_rating.loc[user].dropna()
    return user_books ,user_books.nlargest(3,keep = "first")

In [22]:
user_books , fav_books = get3largest_values(users['User-ID'].sample().tolist()[0])
# print 10 books rated from user 
user_books = user_books.reset_index()
user_books["Book-Title"] = user_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
print(user_books)
# print favourite books 
print("Favourite books: ")
fav_books = fav_books.reset_index()
fav_books["Book-Title"] = fav_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
print(fav_books)

          ISBN  49212                                         Book-Title
0   0060936363    0.0                        Portrait in Sepia : A Novel
1   006440188X   10.0                                  The Secret Garden
2   0312201656    4.0                               I Capture the Castle
3   0345348109    0.0                                  The Killer Angels
4   0345370775    8.0                                      Jurassic Park
5   0375422307    9.0  Persepolis : The Story of a Childhood (Alex Aw...
6   0380724987    0.0  Justice (Peter Decker   Rina Lazarus Novels (P...
7   0425154092    8.0                                From Potter's Field
8   0440212723    0.0                                    Cold Sassy Tree
9   0440213525    8.0                                         The Client
10  044022165X    0.0                                      The Rainmaker
11  0446315176    9.0                      An Unsuitable Job for a Woman
12  0446363251    6.0  Scarlett : The Sequel to Mar

Μέθοδος που δημιουργεί ένα “προφίλ” προτιμήσεων του χρήστη με βάση την
ένωση των λέξεων κλειδιών των 3 βιβλίων, τους συγγραφείς τους και το έτος
έκδοσης τους και επιστρέφει όλα αυτά και τα ISBN των βιβλίων που έχει βαθμολογήσει.

In [23]:
def make_user_profile(user):
    # get  readed and favourites books of user  
    user_books , fav_books = get3largest_values(user)
    
    # initialize 
    keywords = []
    authors = [] 
    years_of_publication = []
    
    for book in fav_books.index:
            keywords += books.set_index('ISBN').loc[book]['KeyWords']
            authors.append(books.set_index('ISBN').loc[book]['Book-Author'])
            years_of_publication.append(books.set_index('ISBN').loc[book]['Year-Of-Publication'])
    return keywords, authors, years_of_publication , user_books

In [24]:
profile = make_user_profile(users['User-ID'].sample().tolist()[0])
print(profile[0],profile[1],profile[2])
user_books  = profile[3]
user_books = user_books.reset_index()
user_books["Book-Title"] = user_books["ISBN"].apply(lambda x : getbooknamefromISBN(x))
user_books

['art', 'decept', 'black', 'ice', 'detect', 'harri', 'bosch', 'mysteri', 'kill', 'shadow', 'st', 'martin', 'minotaur', 'mysteri'] ['Ridley Pearson', 'Michael Connelly', 'Val McDermid'] ['2003', '1996', '2002']


,ISBN,31822,Book-Title
0,0312952813,8.0,The Black Ice (Detective Harry Bosch Mysteries)
1,0312983387,0.0,Killing the Shadows (St. Martin's Minotaur Mys...
2,0425185168,0.0,Forty Words for Sorrow
3,0440222826,0.0,The Mercy Rule
4,0451178017,0.0,Over the Edge
5,0515128821,0.0,The Breaker
6,0786890002,9.0,The Art of Deception


H ομοιότητα Jaccard βασισμένη στον τύπο $ J(X,Y) = |X∩Y| / |X∪Y| $ 

In [25]:
def jaccard_similarity(x,y): 
    intersection_cardinality = len(set(x) & set(y))
    union_cardinality = len(set(x) | set(y))
    return intersection_cardinality/float(union_cardinality)

In [26]:
jaccard_similarity(["hallo","world"] ,["hello","world"])

0.3333333333333333

Έλεγχος για το αν υλοποιήθηκε σωστά.

In [27]:
j = Jaccard()
j.get_raw_score(["hallo","world"] ,["hello","world"])

0.3333333333333333

H ομοιότητα Dice coefficient βασισμένη στον τύπο $DSC(X,Y) = 2|X∩Y| / (|X|+|Y|)$

In [28]:
def dice_coefficient(a, b):
    """dice coefficient 2nt/(na + nb)."""
    a_bigrams = set(a)
    b_bigrams = set(b)
    overlap = len(a_bigrams & b_bigrams)
    return overlap * 2.0/(len(a_bigrams) + len(b_bigrams))

In [29]:
dice_coefficient(["hallo","world"] ,["hello","world"])

0.5

Έλεγχος για το αν υλοποιήθηκε σωστά 

In [30]:
dice = Dice()
dice.get_raw_score(["hallo","world"] ,["hello","world"])

0.5

Μέθοδος που υπολογίζει την ομοιότητα ενός βιβλίου με το “προφίλ” προτιμήσεων ενός χρήστη και ανάλογα το mode που θα επιλέξουμε επιστρέφει την ομοιότητα.(για mode = 1 η ομοιότητά υπολογίζεται ως εξής : Jaccard για τις λέξεις κλειδιά με βάρος 0.2, ισότητα για τους συγγραφείς 0.4,διαφορά ετών έκδοσης 0.4 και για mode = 2 : Dice coefficient για τις λέξεις κλειδιά με βάρος 0.5, ισότητα για τους συγγραφείς 0.3, διαφορά ετών έκδοσης 0.2)


In [31]:
def calc_similarity(user_profile,book,mode):
    # get keywords 
    profile_keywords = user_profile[0]
    book_keywords = book['KeyWords']
    
    # get author 
    profile_authors =  user_profile[1]
    book_author = book['Book-Author']
    
    # get years 
    profile_years =  user_profile[2]
    book_year = book['Year-Of-Publication']
    
    # calculate keywords_similarity 
    if mode == 1:
        keywords_similarity = jaccard_similarity(profile_keywords,book_keywords)
    elif mode == 2:
        keywords_similarity = dice_coefficient(profile_keywords,book_keywords)
    else:
        print("mode is missing or is incoreect")
        pass
    # calculate author similarity 
    if book_author in profile_authors:
        author_similarity = 1 
    else:
        author_similarity = 0
    
    # calculate years similarity
    year_similarity = min([1-(abs(int(x)-int(book_year))/2005) for x in profile_years])
    
    if mode == 1:
        return 0.2 * keywords_similarity + 0.4 * author_similarity + 0.4 * year_similarity
    elif mode==2:
        return 0.5 * keywords_similarity + 0.3 * author_similarity + 0.2 * year_similarity
    

In [32]:
# calculate similarity between profile that already have and random book
location = books['ISBN'].sample().tolist()[0]
book = books.loc[books.sample().index.tolist()[0]]
print(book)
print("Similarity(mode=1) : ",calc_similarity(profile,book,1))
print("Similarity(mode=2) : ",calc_similarity(profile,book,2))

ISBN                                                          0312982011
Book-Title             Abandoned Prayers: The Shocking True Story of ...
Book-Author                                                  Gregg Olsen
Year-Of-Publication                                                 2002
KeyWords               [abandon, prayer, shock, true, stori, obsess, ...
Name: 365238, dtype: object
Similarity(mode=1) :  0.39880299251870327
Similarity(mode=2) :  0.19940149625935163


Μέθοδος που προτείνει στον χρήστη τα 10 αντικείμενα (που δεν έχει βαθμολογήσει) τα οποία
είναι ομοιότερα με το προφίλ του, διατεταγμένα από το περισσότερο προς το
λιγότερο όμοιο.

In [33]:
def recommend(user_profile,mode):
    books_similarity = pd.DataFrame(books['ISBN'])
    books_similarity['similarity'] = None 
    for index,book in books.iterrows():
        if book['ISBN'] not in user_profile[3].index:
            books_similarity.loc[index]['similarity'] =  calc_similarity(user_profile,book,mode)
    
    return books_similarity.sort_values(['similarity'],ascending=False).dropna().head(10)

In [34]:
# calculate and print recomended books with first mode  
recomended_books1 = recommend(profile,1)
recomended_books1["Book-Title"] = recomended_books1["ISBN"].apply(lambda x : getbooknamefromISBN(x))
recomended_books1

,ISBN,similarity,Book-Title
150630,0312950489,0.869434,The Black Echo (Detective Harry Bosch Mysteries)
209582,0316152196,0.852535,Angels Flight (Detective Harry Bosch Mysteries)
183826,0446607274,0.852535,Angels Flight (Detective Harry Bosch Mysteries)
301603,0312963297,0.852336,Trunk Music (Detective Harry Bosch Mysteries)
351556,0786867248,0.829572,The Art of Deception
333481,0446613444,0.829373,The Black Ice
205084,0312955006,0.823404,The Concrete Blonde (A Harry Bosch Novel)
214761,0446612731,0.813089,The Black Echo
257972,0312983603,0.808803,The Mermaids Singing (A Dr. Tony Hill Carol ...
68302,0312983654,0.808803,The Wire in the Blood (A Dr. Tony Hill Carol...


In [35]:
# calculate and print recomended books with second mode  
recomended_books2 = recommend(profile,2)
recomended_books2["Book-Title"] = recomended_books2["ISBN"].apply(lambda x : getbooknamefromISBN(x))
recomended_books2

,ISBN,similarity,Book-Title
150630,0312950489,0.76216,The Black Echo (Detective Harry Bosch Mysteries)
209582,0316152196,0.710127,Angels Flight (Detective Harry Bosch Mysteries)
183826,0446607274,0.710127,Angels Flight (Detective Harry Bosch Mysteries)
301603,0312963297,0.710028,Trunk Music (Detective Harry Bosch Mysteries)
351556,0786867248,0.632735,The Art of Deception
333481,0446613444,0.632635,The Black Ice
205084,0312955006,0.610313,The Concrete Blonde (A Harry Bosch Novel)
214761,0446612731,0.566068,The Black Echo
257972,0312983603,0.547021,The Mermaids Singing (A Dr. Tony Hill Carol ...
68302,0312983654,0.547021,The Wire in the Blood (A Dr. Tony Hill Carol...


Εδω ξεκινάει το πείραμα.Επιλέγουμε τυχαία 5 χρήστες και παράγουμε για
αυτούς τις σχετικές προτάσεις και με τις δύο εναλλακτικές, οι οποίες και θα
καταγράφουν σε αντίστοιχα αρχεία εξόδου (διαφορετικά).

In [133]:
def make_predictions():
    # random sample of 5 users 
    sampled_users = users['User-ID'].sample(5).tolist()

    for user in sampled_users:
        # make user's profile 
        profile = make_user_profile(user)

        # recomend 10 books with first method
        recomended_books1 = recommend(profile,1)
        recomended_books1["Book-Title"] = recomended_books1["ISBN"].apply(lambda x : getbooknamefromISBN(x))

        # recomend 10 books with the second method
        recomended_books2 = recommend(profile,2)
        recomended_books2["Book-Title"] = recomended_books2["ISBN"].apply(lambda x : getbooknamefromISBN(x))

        # save them 
        recomended_books1.to_csv(str(user)+'_1.csv',index= None,encoding='cp437')
        recomended_books2.to_csv(str(user)+'_2.csv',index= None,encoding='cp437')
    
    # save sampled users 
    with open('sampled_users.csv', 'w', newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        wr.writerow(sampled_users)
        

In [134]:
make_predictions()

Για κάθε χρήστη που τυχαία επέλεξε προηγουμένως, μετραει την ομοιότητα των λιστών με τα αποτέλεσματα
υπολογίζοντας τη μέση επικάλυψη τους και τα επιστρέφει μαζί με 2 λίστες που έχουν τα αποτελέσματα των χρηστών.Πρωτα πρέπει να φτιαχτεί μια μέθοδος για να μετράμε την επικάλυψη βάση του τύπου $overlap_coefficient(X,Y)=|X∩Y|/min(|X|,|Y|)$

In [38]:
def overlap(x,y):
    intersection_cardinality = len(set(x) & set(y))
    return intersection_cardinality / min(len(set(x)),len(set(y)))

In [39]:
overlap(["hallo","world"] ,["hello","world"])

0.5

Έλεγχος για το αν υλοποιήθηκε σωστά. 

In [40]:
oc = OverlapCoefficient()
oc.get_raw_score(["hallo","world"] ,["hello","world"])

0.5

In [141]:
def read_sampled_users():
    sampled_users = [] 
    with open('sampled_users.csv', 'r') as f:
        reader = csv.reader(f)
        sampled_users = list(reader)[0]
    return sampled_users

In [231]:
# read the User-ID fro sampled users  
sampled_users = read_sampled_users()
print(sampled_users)

['77585', '142005', '135753', '10660', '232964']


In [232]:
def read_recomended_books(sampled_users):
    # lists with recomended books for every user 
    recomended_books1_list = []
    recomended_books2_list = [] 
    
    for user in sampled_users:
        # read recomended_books with first method
        recomended_books1 = pd.read_csv(str(user) + '_1.csv',low_memory=False,encoding='cp437',dtype={'ISBN': str})
        recomended_books1_list.append(recomended_books1)
        # read recomended_books with second method
        recomended_books2 = pd.read_csv(str(user) + '_2.csv',low_memory=False,encoding='cp437',dtype={'ISBN': str})
        recomended_books2_list.append(recomended_books2)
    
    return recomended_books1_list,recomended_books2_list

In [233]:
# read reconmended books
recomended_books1_list , recomended_books2_list = read_recomended_books(sampled_users)
# print first sampled user recomended books 
print(recomended_books1_list[0])
print(recomended_books2_list[0])

         ISBN  similarity                              Book-Title
0  0345410017    0.855347                          Fahrenheit 451
1  0312169787    0.848005                  The Red Tent : A Novel
2  0345380371    0.798404                              Rising Sun
3  0345378482    0.798404                    The Andromeda Strain
4  0394588169    0.798005                           Jurassic Park
5  0345354648    0.797805                           Five Patients
6  0345354613    0.797606                      Eaters of the Dead
7  0345353145    0.797606                                  Sphere
8  0743225325    0.797207                    Good Harbor: A Novel
9  0553277537    0.797007  Dandelion Wine (Grand Master Editions)
         ISBN  similarity                              Book-Title
0  0345410017    0.721324                          Fahrenheit 451
1  0312169787    0.699002                  The Red Tent : A Novel
2  0345380371    0.499202                              Rising Sun
3  0345378

In [234]:
def calc_weighted_overlap(recomended_books1,recomended_books2):
    weighted_overlap = 0
    sum_overlap = 0 
    for i in range(10):
        # get isbn
        isbn1 = recomended_books1.iloc[i, 0] 
        isbn2 = recomended_books2.iloc[i, 0]
        
        # get books 
        book1 = books.set_index('ISBN').loc[str(isbn1)]
        book2 = books.set_index('ISBN').loc[str(isbn2)]

        # make lists with books charachteristics 
        book1_ch = book1['KeyWords'] + [book1['Book-Author']] + [book1['Year-Of-Publication']]
        book2_ch = book2['KeyWords'] + [book2['Book-Author']] + [book2['Year-Of-Publication']]
        # update sum overlap 
        sum_overlap += overlap(book1_ch,book2_ch)
    
    # compute weighted_overlap
    weighted_overlap = sum_overlap/10

    return weighted_overlap

In [235]:
# calculate weighted_overlaps for every user  
weighted_overlaps = []       

for i in range(5):    
    weighted_overlap = calc_weighted_overlap(recomended_books1_list[i],recomended_books2_list[i])
    weighted_overlaps.append(weighted_overlap)

print('Weighted_Overlaps')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps[i])

Weighted_Overlaps
User:  77585   1.0
User:  142005   0.925
User:  135753   0.9333333333333333
User:  10660   1.0
User:  232964   0.975


Για κάθε χρήστη, υπολογίζει σε πόσες λίστες εμφανίζεται το κάθε αποτέλεσμα και
ταξινομει τα σε μία νέα λίστα—πρώτα με βάση του πλήθους εμφανίσεων και
μετά με βάση την ομοιότητα με το προφίλ του χρήστη.

In [198]:
def count_occurancies(recomended_book,recomended_books_list):
    count = 0 
    for books in recomended_books_list:
        if recomended_book['ISBN'] in books['ISBN'].tolist():
            count += 1
    return count

Μέθοδος δημιουργίας golden standard δωσμένου μιας λίστας ενος χρήστη και την λίστα με τις λίστες όλων των χρηστών.

In [238]:
def make_golden_standard(recomended_books,recomended_books_list):
    counts = []
    for index,book in recomended_books.iterrows():
        counts.append(count_occurancies(book,recomended_books_list))
    recomended_books['count'] = counts
    golden_standard = recomended_books.sort_values(by=['count','similarity'],ascending=False)
    
    return golden_standard

In [254]:
# create golden standards and save them to cvs 
golden_standards1 = []
golden_standards2 = []
for i in range(5):
    # golden standard with Jaccard 
    golden_standard1 = make_golden_standard(recomended_books1_list[i],recomended_books1_list)
    golden_standards1.append(golden_standard1)
    
    # golden standard with Dice
    golden_standard2 = make_golden_standard(recomended_books2_list[i],recomended_books2_list)
    golden_standards2.append(golden_standard2)
    
    # save them 
    golden_standard1.to_csv('golden' + str(sampled_users[i])+'_1.csv',index= None,encoding='cp437')
    golden_standard2.to_csv('golden' + str(sampled_users[i])+'_2.csv',index= None,encoding='cp437')

Eπανάλάψη του βήματος 2 συγκρίνει όμως κάθε λίστα
με την golden_standard που παράχθηκε (μέχρι το 10 της στοιχείο).

In [257]:
weighted_overlaps1 = []       
weighted_overlaps2 = []    
for i in range(5):    
    weighted_overlap = calc_weighted_overlap(recomended_books1_list[i],golden_standards1[i])
    weighted_overlaps1.append(weighted_overlap)
    
    weighted_overlap = calc_weighted_overlap(recomended_books2_list[i],golden_standards2[i])
    weighted_overlaps2.append(weighted_overlap)

print('Weighted_Overlaps_1')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps1[i])
    
print('\nWeighted_Overlaps_2')
for i in range(5):
    print('User: ',sampled_users[i]," ",weighted_overlaps2[i])

Weighted_Overlaps_1
User:  77585   1.0
User:  142005   1.0
User:  135753   0.575
User:  10660   1.0
User:  232964   0.3678571428571429

Weighted_Overlaps_2
User:  77585   1.0
User:  142005   1.0
User:  135753   0.575
User:  10660   1.0
User:  232964   0.3678571428571429
